In [1]:
import pandas as pd
import holidays
from pyrosm import OSM
import geopandas as gpd
import gc
import pyarrow
import numpy as np
import pyarrow.parquet as pq

# Notebook Criado com o intuito de extrair, tratar e incrementar a base de dados utilizada

In [2]:
df = pd.read_csv('../data_raw/sinistros_2022-2025.csv', sep=';', encoding='latin1', low_memory = False)

In [3]:
mask = df['logradouro'].str.contains('SP 348|SP 280|SP 070|SP 330|SP 310|SP 150|SP 123|SP 270', case=False, regex=True, na=False)

df = df[mask]

In [4]:
df.loc[df['logradouro'].str.contains('348', case=False), 'logradouro'] = 'SP 348'
df.loc[df['logradouro'].str.contains('280', case=False), 'logradouro'] = 'SP 280'
df.loc[df['logradouro'].str.contains('070', case=False), 'logradouro'] = 'SP 070'
df.loc[df['logradouro'].str.contains('330', case=False), 'logradouro'] = 'SP 330'
df.loc[df['logradouro'].str.contains('310', case=False), 'logradouro'] = 'SP 310'
df.loc[df['logradouro'].str.contains('150', case=False), 'logradouro'] = 'SP 150'
df.loc[df['logradouro'].str.contains('123', case=False), 'logradouro'] = 'SP 123'
df.loc[df['logradouro'].str.contains('270', case=False), 'logradouro'] = 'SP 270'

print(df['logradouro'].value_counts())


logradouro
SP 270    7379
SP 330    6748
SP 280    4314
SP 310    3315
SP 150    2781
SP 348    2679
SP 070    2292
SP 123     457
Name: count, dtype: int64


In [5]:
df = df[ 
    ['data_sinistro', 'hora_sinistro', 'logradouro', 'municipio', 'latitude', 'longitude', 'tipo_registro', 'turno', 
     'tp_sinistro_primario', 'qtd_gravidade_fatal', 'qtd_gravidade_grave', 'qtd_gravidade_leve', 'qtd_gravidade_ileso', 
     'qtd_caminhao', 'qtd_motocicleta', 'qtd_automovel', 'qtd_pedestre', 'qtd_bicicleta', 'qtd_onibus', 'qtd_veic_outros', 
     'qtd_veic_nao_disponivel', 'conservacao', 'numero_logradouro', 'tp_sinistro_atropelamento', 'tp_sinistro_colisao_frontal', 
    'tp_sinistro_colisao_lateral', 'tp_sinistro_colisao_transversal', 'tp_sinistro_colisao_outros', 
     'tp_sinistro_choque', 'tp_sinistro_capotamento', 'tp_sinistro_engavetamento', 'tp_sinistro_tombamento', 'tp_sinistro_outros']
    ].copy()

In [6]:
df.loc[df['conservacao'].str.contains(r'\d+', regex=True, na=False), 'conservacao'] = 'DER'

df = df.rename(columns={'conservacao': 'concessionaria'})

In [7]:
print(df['concessionaria'].value_counts(dropna=False))

concessionaria
DER                            10824
AUTOBAN                         5141
NAO DISPONIVEL                  4843
ECOPISTAS                       2289
VIAOESTE                        1539
ECONOROESTE                     1127
ECOVIAS                          937
CART                             861
EIXOSP - PIPA                    576
ECOVIAS RAPOSO-CASTELO           362
INTERVIAS                        305
SPVIAS                           304
ENTREVIAS                        211
NaN                              209
ROTA SOROCABANA                  197
COLINAS                          171
VIAPAULISTA                       62
PREFEITURA                         6
AUTOPISTA REGIS BITTENCOURT        1
Name: count, dtype: int64


In [8]:
df.dropna(subset=['concessionaria'], inplace=True)

In [9]:
df.loc[df['concessionaria'].str.contains('PREFEITURA', na=False), 'concessionaria'] = 'DER'

df = df[~df['concessionaria'].str.contains('REGIS', na=False)].copy()

In [10]:
gps_cols = ['latitude', 'longitude', 'numero_logradouro']

for col in gps_cols:

    df[col] = df[col].astype(str).str.replace(',', '.')
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [11]:
df['numero_logradouro'].describe()

count      29736.000000
mean         583.552113
std        10125.207116
min            0.000000
25%           23.000000
50%           63.000000
75%          160.000000
max      1000000.000000
Name: numero_logradouro, dtype: float64

In [12]:
df.loc[df['numero_logradouro'] > 700, 'numero_logradouro'] = None

df['numero_logradouro'].describe()

count    29211.000000
mean       116.608251
std        137.345143
min          0.000000
25%         23.000000
50%         61.000000
75%        147.000000
max        700.000000
Name: numero_logradouro, dtype: float64

In [13]:
df['data_hora'] = pd.to_datetime(df['data_sinistro'] + ' '+ df['hora_sinistro'], format='%d/%m/%Y %H:%M', errors='coerce')

In [14]:
df['data_hora'].info()
df['data_hora'].isna().sum()
df['data_hora'].min()
df['data_hora'].max()

<class 'pandas.core.series.Series'>
Index: 29755 entries, 11 to 755855
Series name: data_hora
Non-Null Count  Dtype         
--------------  -----         
29732 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 464.9 KB


Timestamp('2025-10-31 23:50:00')

In [15]:
df.dropna(subset=['data_hora'], inplace=True)

df['data_hora'].isna().sum()

np.int64(0)

In [16]:
df = df.drop(['data_sinistro', 'hora_sinistro'], axis=1)

In [17]:
df['hora'] = df['data_hora'].dt.hour
df['mes'] = df['data_hora'].dt.month
df['dia_semana'] = df['data_hora'].dt.dayofweek

In [18]:
df.columns

Index(['logradouro', 'municipio', 'latitude', 'longitude', 'tipo_registro',
       'turno', 'tp_sinistro_primario', 'qtd_gravidade_fatal',
       'qtd_gravidade_grave', 'qtd_gravidade_leve', 'qtd_gravidade_ileso',
       'qtd_caminhao', 'qtd_motocicleta', 'qtd_automovel', 'qtd_pedestre',
       'qtd_bicicleta', 'qtd_onibus', 'qtd_veic_outros',
       'qtd_veic_nao_disponivel', 'concessionaria', 'numero_logradouro',
       'tp_sinistro_atropelamento', 'tp_sinistro_colisao_frontal',
       'tp_sinistro_colisao_lateral', 'tp_sinistro_colisao_transversal',
       'tp_sinistro_colisao_outros', 'tp_sinistro_choque',
       'tp_sinistro_capotamento', 'tp_sinistro_engavetamento',
       'tp_sinistro_tombamento', 'tp_sinistro_outros', 'data_hora', 'hora',
       'mes', 'dia_semana'],
      dtype='object')

In [19]:
date_cols = ['hora', 'mes', 'dia_semana']

df[date_cols].apply(pd.unique)

hora          [17, 23, 10, 11, 13, 15, 3, 7, 4, 14, 18, 19, ...
mes                     [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
dia_semana                                [5, 6, 0, 1, 2, 3, 4]
dtype: object

In [20]:
df['municipio'].value_counts()

municipio
SAO PAULO                3635
COTIA                    1720
CAMPINAS                 1638
SAO BERNARDO DO CAMPO    1514
JUNDIAI                  1282
                         ... 
ITATIBA                     1
BROTAS                      1
BALSAMO                     1
TAPIRATIBA                  1
GUAPIACU                    1
Name: count, Length: 214, dtype: int64

In [21]:
df['municipio'] = df['municipio'].str.strip()
df['municipio'] = df['municipio'].str.upper()

In [22]:
df[['latitude', 'longitude']].describe()

,latitude,longitude
count,29695.000000,29695.000000
mean,-273.329896,-589.939808
std,7558.845529,15987.929701
min,-236197.000000,-513486.000000
25%,-23.531369,-47.586299
50%,-23.393236,-47.017820
75%,-22.785220,-46.768778
max,-19.978126,0.000000


In [23]:
df = df[(df['latitude'] >= -26) & (df['latitude'] <= -19) & (df['longitude'] >= -54) & (df['longitude'] <= -44)]

df.shape

(29659, 35)

In [24]:
df['tipo_registro'].value_counts()

tipo_registro
SINISTRO NAO FATAL    21592
NOTIFICACAO            6190
SINISTRO FATAL         1877
Name: count, dtype: int64

In [25]:
df['turno'].value_counts()

turno
MANHA        9110
TARDE        8637
NOITE        8088
MADRUGADA    3824
Name: count, dtype: int64

In [26]:
df['tp_sinistro_primario'].value_counts()

tp_sinistro_primario
COLISAO           11630
OUTROS            10146
CHOQUE             4835
NAO DISPONIVEL     1703
ATROPELAMENTO      1345
Name: count, dtype: int64

In [27]:
qtd_cols = ['qtd_gravidade_fatal', 'qtd_gravidade_grave', 'qtd_gravidade_leve', 'qtd_gravidade_ileso',
    'qtd_caminhao', 'qtd_motocicleta', 'qtd_automovel', 'qtd_pedestre', 'qtd_bicicleta', 'qtd_onibus', 
    'qtd_veic_outros', 'qtd_veic_nao_disponivel']

df[qtd_cols].describe()

,qtd_gravidade_fatal,qtd_gravidade_grave,qtd_gravidade_leve,qtd_gravidade_ileso,qtd_caminhao,qtd_motocicleta,qtd_automovel,qtd_pedestre,qtd_bicicleta,qtd_onibus,qtd_veic_outros,qtd_veic_nao_disponivel
count,1877.000000,3668.000000,17089.000000,0.0,4096.000000,12370.000000,14028.000000,465.000000,315.000000,464.000000,719.000000,587.000000
mean,1.073522,1.106052,1.279829,NaN,1.241211,1.055214,1.301183,1.012903,1.003175,1.030172,1.011127,1.398637
std,0.321456,0.431177,0.696007,NaN,0.491114,0.236066,0.678658,0.146236,0.056344,0.194845,0.104967,0.744398
min,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
max,5.000000,12.000000,20.000000,NaN,5.000000,3.000000,9.000000,3.000000,2.000000,3.000000,2.000000,9.000000


In [28]:
df['qtd_gravidade_ileso'].info()

<class 'pandas.core.series.Series'>
Index: 29659 entries, 11 to 755855
Series name: qtd_gravidade_ileso
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 463.4 KB


In [29]:
df = df.drop('qtd_gravidade_ileso', axis=1)

In [30]:
qtd_cols = ['qtd_gravidade_fatal', 'qtd_gravidade_grave', 'qtd_gravidade_leve',
    'qtd_caminhao', 'qtd_motocicleta', 'qtd_automovel', 'qtd_pedestre', 'qtd_bicicleta', 
    'qtd_onibus', 'qtd_veic_outros', 'qtd_veic_nao_disponivel']

df[qtd_cols].apply(pd.unique)

qtd_gravidade_fatal                           [nan, 1.0, 3.0, 2.0, 4.0, 5.0]
qtd_gravidade_grave        [nan, 1.0, 2.0, 3.0, 4.0, 6.0, 8.0, 12.0, 11.0...
qtd_gravidade_leve         [nan, 4.0, 1.0, 2.0, 3.0, 5.0, 6.0, 8.0, 7.0, ...
qtd_caminhao                                  [nan, 1.0, 2.0, 3.0, 5.0, 4.0]
qtd_motocicleta                                         [nan, 1.0, 2.0, 3.0]
qtd_automovel              [nan, 2.0, 1.0, 5.0, 3.0, 4.0, 6.0, 9.0, 7.0, ...
qtd_pedestre                                            [nan, 1.0, 3.0, 2.0]
qtd_bicicleta                                                [nan, 1.0, 2.0]
qtd_onibus                                              [nan, 1.0, 2.0, 3.0]
qtd_veic_outros                                              [nan, 1.0, 2.0]
qtd_veic_nao_disponivel             [nan, 1.0, 2.0, 4.0, 3.0, 6.0, 9.0, 5.0]
dtype: object

In [31]:
df[qtd_cols] = df[qtd_cols].fillna(0)

In [32]:
df[qtd_cols].apply(pd.unique)

qtd_gravidade_fatal                           [0.0, 1.0, 3.0, 2.0, 4.0, 5.0]
qtd_gravidade_grave        [0.0, 1.0, 2.0, 3.0, 4.0, 6.0, 8.0, 12.0, 11.0...
qtd_gravidade_leve         [0.0, 4.0, 1.0, 2.0, 3.0, 5.0, 6.0, 8.0, 7.0, ...
qtd_caminhao                                  [0.0, 1.0, 2.0, 3.0, 5.0, 4.0]
qtd_motocicleta                                         [0.0, 1.0, 2.0, 3.0]
qtd_automovel              [0.0, 2.0, 1.0, 5.0, 3.0, 4.0, 6.0, 9.0, 7.0, ...
qtd_pedestre                                            [0.0, 1.0, 3.0, 2.0]
qtd_bicicleta                                                [0.0, 1.0, 2.0]
qtd_onibus                                              [0.0, 1.0, 2.0, 3.0]
qtd_veic_outros                                              [0.0, 1.0, 2.0]
qtd_veic_nao_disponivel             [0.0, 1.0, 2.0, 4.0, 3.0, 6.0, 9.0, 5.0]
dtype: object

In [33]:
df[qtd_cols].describe()

,qtd_gravidade_fatal,qtd_gravidade_grave,qtd_gravidade_leve,qtd_caminhao,qtd_motocicleta,qtd_automovel,qtd_pedestre,qtd_bicicleta,qtd_onibus,qtd_veic_outros,qtd_veic_nao_disponivel
count,29659.000000,29659.000000,29659.000000,29659.000000,29659.000000,29659.000000,29659.000000,29659.000000,29659.000000,29659.000000,29659.000000
mean,0.067939,0.136788,0.737415,0.171415,0.440102,0.615429,0.015881,0.010654,0.016117,0.024512,0.027681
std,0.273600,0.394430,0.824080,0.465497,0.542183,0.799925,0.127155,0.102999,0.130139,0.156369,0.221133
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,12.000000,20.000000,5.000000,3.000000,9.000000,3.000000,2.000000,3.000000,2.000000,9.000000


In [34]:
df[qtd_cols] = df[qtd_cols].astype('int')

In [35]:
mask = (df['qtd_gravidade_fatal'] + df['qtd_gravidade_grave'] + df['qtd_gravidade_leve'] > 0) & (df['qtd_caminhao'] + df['qtd_motocicleta'] + df['qtd_automovel'] + df['qtd_pedestre'] + df['qtd_bicicleta'] + df['qtd_onibus'] + df['qtd_veic_outros'] + df['qtd_veic_nao_disponivel'] == 0)

len(df[mask])

134

In [36]:
df = df[~mask]

In [37]:
df[qtd_cols].describe()

,qtd_gravidade_fatal,qtd_gravidade_grave,qtd_gravidade_leve,qtd_caminhao,qtd_motocicleta,qtd_automovel,qtd_pedestre,qtd_bicicleta,qtd_onibus,qtd_veic_outros,qtd_veic_nao_disponivel
count,29525.000000,29525.000000,29525.000000,29525.000000,29525.000000,29525.000000,29525.000000,29525.000000,29525.00000,29525.000000,29525.000000
mean,0.068146,0.135749,0.737206,0.172193,0.442100,0.618222,0.015953,0.010703,0.01619,0.024623,0.027807
std,0.274021,0.393601,0.824532,0.466409,0.542599,0.800661,0.127438,0.103230,0.13043,0.156715,0.221627
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000
max,5.000000,12.000000,20.000000,5.000000,3.000000,9.000000,3.000000,2.000000,3.00000,2.000000,9.000000


In [38]:
print(df[(df['qtd_gravidade_leve'] == 20)].T)

                                              105373               193082  \
logradouro                                    SP 348               SP 348   
municipio                                    JUNDIAI            SAO PAULO   
latitude                                  -23.194158           -23.414878   
longitude                                 -46.929913           -46.765329   
tipo_registro                     SINISTRO NAO FATAL   SINISTRO NAO FATAL   
turno                                          MANHA            MADRUGADA   
tp_sinistro_primario                          CHOQUE              COLISAO   
qtd_gravidade_fatal                                0                    0   
qtd_gravidade_grave                                2                    0   
qtd_gravidade_leve                                20                   20   
qtd_caminhao                                       0                    1   
qtd_motocicleta                                    0                    0   

In [39]:
print(df[(df['qtd_gravidade_grave'] == 12)].T)

                                                  410834
logradouro                                        SP 280
municipio                        SANTA CRUZ DO RIO PARDO
latitude                                      -22.909219
longitude                                     -49.620055
tipo_registro                         SINISTRO NAO FATAL
turno                                              TARDE
tp_sinistro_primario                              CHOQUE
qtd_gravidade_fatal                                    0
qtd_gravidade_grave                                   12
qtd_gravidade_leve                                    12
qtd_caminhao                                           0
qtd_motocicleta                                        0
qtd_automovel                                          0
qtd_pedestre                                           0
qtd_bicicleta                                          0
qtd_onibus                                             1
qtd_veic_outros                

In [40]:
print(df[(df['qtd_automovel'] == 9)].T)

                                              133757               174705
logradouro                                    SP 150               SP 348
municipio                                  SAO PAULO             CAIEIRAS
latitude                                  -23.639908           -23.349684
longitude                                 -46.592185           -46.808828
tipo_registro                     SINISTRO NAO FATAL   SINISTRO NAO FATAL
turno                                          NOITE                TARDE
tp_sinistro_primario                          OUTROS               OUTROS
qtd_gravidade_fatal                                0                    0
qtd_gravidade_grave                                0                    0
qtd_gravidade_leve                                 3                    1
qtd_caminhao                                       2                    0
qtd_motocicleta                                    0                    0
qtd_automovel                         

In [41]:
print(df[(df['qtd_caminhao'] == 5)].T)

                                              255631
logradouro                                    SP 330
municipio                                   CAMPINAS
latitude                                    -21.6787
longitude                                 -47.605022
tipo_registro                     SINISTRO NAO FATAL
turno                                          MANHA
tp_sinistro_primario                          OUTROS
qtd_gravidade_fatal                                0
qtd_gravidade_grave                                0
qtd_gravidade_leve                                 1
qtd_caminhao                                       5
qtd_motocicleta                                    0
qtd_automovel                                      0
qtd_pedestre                                       0
qtd_bicicleta                                      0
qtd_onibus                                         0
qtd_veic_outros                                    0
qtd_veic_nao_disponivel                       

In [42]:
print(df[(df['qtd_veic_nao_disponivel'] == 9)].T)

                                                351097
logradouro                                      SP 150
municipio                        SAO BERNARDO DO CAMPO
latitude                                    -23.788529
longitude                                   -46.520423
tipo_registro                       SINISTRO NAO FATAL
turno                                            MANHA
tp_sinistro_primario                            OUTROS
qtd_gravidade_fatal                                  0
qtd_gravidade_grave                                  0
qtd_gravidade_leve                                   0
qtd_caminhao                                         0
qtd_motocicleta                                      0
qtd_automovel                                        0
qtd_pedestre                                         0
qtd_bicicleta                                        0
qtd_onibus                                           0
qtd_veic_outros                                      0
qtd_veic_n

In [43]:
mask = (df['qtd_veic_nao_disponivel'] == 9) & (df['qtd_gravidade_fatal'] + df['qtd_gravidade_grave'] + df['qtd_gravidade_leve'] == 0)

df = df[~mask]

In [44]:
df[qtd_cols].describe()

,qtd_gravidade_fatal,qtd_gravidade_grave,qtd_gravidade_leve,qtd_caminhao,qtd_motocicleta,qtd_automovel,qtd_pedestre,qtd_bicicleta,qtd_onibus,qtd_veic_outros,qtd_veic_nao_disponivel
count,29524.000000,29524.000000,29524.000000,29524.000000,29524.000000,29524.000000,29524.000000,29524.000000,29524.000000,29524.000000,29524.000000
mean,0.068148,0.135754,0.737231,0.172199,0.442115,0.618243,0.015953,0.010703,0.016190,0.024624,0.027503
std,0.274026,0.393607,0.824535,0.466416,0.542602,0.800666,0.127441,0.103231,0.130432,0.156718,0.215391
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,12.000000,20.000000,5.000000,3.000000,9.000000,3.000000,2.000000,3.000000,2.000000,6.000000


In [45]:
print(df[(df['qtd_veic_nao_disponivel'] == 6)].T)

                                               45070
logradouro                                    SP 270
municipio                        PRESIDENTE PRUDENTE
latitude                                     -22.136
longitude                                   -51.4154
tipo_registro                     SINISTRO NAO FATAL
turno                                          TARDE
tp_sinistro_primario                          OUTROS
qtd_gravidade_fatal                                0
qtd_gravidade_grave                                0
qtd_gravidade_leve                                 0
qtd_caminhao                                       0
qtd_motocicleta                                    0
qtd_automovel                                      0
qtd_pedestre                                       0
qtd_bicicleta                                      0
qtd_onibus                                         0
qtd_veic_outros                                    0
qtd_veic_nao_disponivel                       

In [46]:
mask = (df['qtd_veic_nao_disponivel'] == 6) & (df['qtd_gravidade_fatal'] + df['qtd_gravidade_grave'] + df['qtd_gravidade_leve'] == 0)

df = df[~mask]

In [47]:
df[qtd_cols].describe()

,qtd_gravidade_fatal,qtd_gravidade_grave,qtd_gravidade_leve,qtd_caminhao,qtd_motocicleta,qtd_automovel,qtd_pedestre,qtd_bicicleta,qtd_onibus,qtd_veic_outros,qtd_veic_nao_disponivel
count,29523.00000,29523.000000,29523.000000,29523.000000,29523.000000,29523.000000,29523.000000,29523.000000,29523.000000,29523.000000,29523.000000
mean,0.06815,0.135759,0.737256,0.172205,0.442130,0.618264,0.015954,0.010704,0.016191,0.024625,0.027301
std,0.27403,0.393613,0.824538,0.466423,0.542605,0.800672,0.127443,0.103233,0.130434,0.156720,0.212571
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.00000,12.000000,20.000000,5.000000,3.000000,9.000000,3.000000,2.000000,3.000000,2.000000,5.000000


In [48]:
print(df[(df['qtd_veic_nao_disponivel'] == 5)].T)

                                              364735               494641
logradouro                                    SP 348               SP 348
municipio                                    VINHEDO              ITUPEVA
latitude                                  -23.097736           -23.107841
longitude                                 -47.008565           -47.009309
tipo_registro                     SINISTRO NAO FATAL   SINISTRO NAO FATAL
turno                                          MANHA                TARDE
tp_sinistro_primario                          OUTROS               OUTROS
qtd_gravidade_fatal                                0                    0
qtd_gravidade_grave                                0                    0
qtd_gravidade_leve                                 0                    0
qtd_caminhao                                       0                    0
qtd_motocicleta                                    0                    0
qtd_automovel                         

In [49]:
mask = (df['qtd_veic_nao_disponivel'] == 5) & (df['qtd_gravidade_fatal'] + df['qtd_gravidade_grave'] + df['qtd_gravidade_leve'] == 0)

df = df[~mask]

In [50]:
df[qtd_cols].describe()

,qtd_gravidade_fatal,qtd_gravidade_grave,qtd_gravidade_leve,qtd_caminhao,qtd_motocicleta,qtd_automovel,qtd_pedestre,qtd_bicicleta,qtd_onibus,qtd_veic_outros,qtd_veic_nao_disponivel
count,29521.000000,29521.000000,29521.000000,29521.000000,29521.000000,29521.000000,29521.000000,29521.000000,29521.000000,29521.000000,29521.000000
mean,0.068155,0.135768,0.737306,0.172216,0.442160,0.618306,0.015955,0.010704,0.016192,0.024627,0.026964
std,0.274039,0.393625,0.824543,0.466436,0.542611,0.800683,0.127447,0.103236,0.130439,0.156726,0.208600
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,12.000000,20.000000,5.000000,3.000000,9.000000,3.000000,2.000000,3.000000,2.000000,4.000000


In [51]:
mask = (df['qtd_veic_nao_disponivel'] == 4) & (df['qtd_gravidade_fatal'] + df['qtd_gravidade_grave'] + df['qtd_gravidade_leve'] == 0)

df = df[~mask]

In [52]:
df[qtd_cols].describe()

,qtd_gravidade_fatal,qtd_gravidade_grave,qtd_gravidade_leve,qtd_caminhao,qtd_motocicleta,qtd_automovel,qtd_pedestre,qtd_bicicleta,qtd_onibus,qtd_veic_outros,qtd_veic_nao_disponivel
count,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000,29516.00000,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000
mean,0.068166,0.135791,0.737431,0.172246,0.442235,0.61841,0.015957,0.010706,0.016195,0.024631,0.026291
std,0.274061,0.393654,0.824557,0.466470,0.542627,0.80071,0.127458,0.103245,0.130450,0.156738,0.202106
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.000000,12.000000,20.000000,5.000000,3.000000,9.00000,3.000000,2.000000,3.000000,2.000000,4.000000


In [53]:
print(df[(df['qtd_veic_nao_disponivel'] == 4)].T)

                                                 36341
logradouro                                      SP 310
municipio                        SAO JOSE DO RIO PRETO
latitude                                    -20.851006
longitude                                   -49.408358
tipo_registro                       SINISTRO NAO FATAL
turno                                            MANHA
tp_sinistro_primario                           COLISAO
qtd_gravidade_fatal                                  0
qtd_gravidade_grave                                  0
qtd_gravidade_leve                                   1
qtd_caminhao                                         0
qtd_motocicleta                                      1
qtd_automovel                                        1
qtd_pedestre                                         0
qtd_bicicleta                                        0
qtd_onibus                                           0
qtd_veic_outros                                      0
qtd_veic_n

In [54]:
accident_type_col = ['tp_sinistro_atropelamento', 'tp_sinistro_colisao_frontal', 
            'tp_sinistro_colisao_lateral', 'tp_sinistro_colisao_transversal', 'tp_sinistro_colisao_outros', 
            'tp_sinistro_choque', 'tp_sinistro_capotamento', 'tp_sinistro_engavetamento', 'tp_sinistro_tombamento', 
            'tp_sinistro_outros']


df[accident_type_col] = df[accident_type_col].fillna(0)
df[accident_type_col] = df[accident_type_col].replace('S', 1)

df[accident_type_col] = df[accident_type_col].astype('int')

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_18908\561765391.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[accident_type_col] = df[accident_type_col].replace('S', 1)


In [55]:
df[accident_type_col].describe()

,tp_sinistro_atropelamento,tp_sinistro_colisao_frontal,tp_sinistro_colisao_lateral,tp_sinistro_colisao_transversal,tp_sinistro_colisao_outros,tp_sinistro_choque,tp_sinistro_capotamento,tp_sinistro_engavetamento,tp_sinistro_tombamento,tp_sinistro_outros
count,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000,29516.000000
mean,0.049634,0.017245,0.141618,0.246443,0.002473,0.200197,0.071419,0.032050,0.346693,0.186306
std,0.217192,0.130185,0.348664,0.430947,0.049671,0.400154,0.257528,0.176137,0.475925,0.389360
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [56]:
print(df[accident_type_col].sum())

tp_sinistro_atropelamento           1465
tp_sinistro_colisao_frontal          509
tp_sinistro_colisao_lateral         4180
tp_sinistro_colisao_transversal     7274
tp_sinistro_colisao_outros            73
tp_sinistro_choque                  5909
tp_sinistro_capotamento             2108
tp_sinistro_engavetamento            946
tp_sinistro_tombamento             10233
tp_sinistro_outros                  5499
dtype: int64


In [57]:
print(len(df[(df['tp_sinistro_tombamento'] == 1) & (df['qtd_motocicleta'] > 0)]))

8967


In [58]:
mask = (df['tp_sinistro_atropelamento'] + df['tp_sinistro_colisao_frontal'] + df['tp_sinistro_colisao_lateral'] + df['tp_sinistro_colisao_transversal'] + df['tp_sinistro_colisao_outros'] + df['tp_sinistro_choque'] + df['tp_sinistro_capotamento'] + df['tp_sinistro_engavetamento'] + df['tp_sinistro_tombamento'] + df['tp_sinistro_outros'] == 0)

print(len(df[mask]))

1702


In [59]:
print(df[mask]['tp_sinistro_primario'].value_counts())

tp_sinistro_primario
NAO DISPONIVEL    1702
Name: count, dtype: int64


In [60]:
df['indice_severidade'] = (df['qtd_gravidade_fatal'] * 250) + (df['qtd_gravidade_grave'] * 30) + (df['qtd_gravidade_leve'] * 1)

df['indice_severidade'] = df['indice_severidade'].astype('int')

print(df['indice_severidade'].describe())

count    29516.000000
mean        21.852758
std         68.323998
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max       1250.000000
Name: indice_severidade, dtype: float64


In [61]:
mask = df['indice_severidade'] == 1250

df[mask].T

,370214
logradouro,SP 280
municipio,BOFETE
latitude,-23.164706
longitude,-48.341383
tipo_registro,SINISTRO FATAL
turno,NOITE
tp_sinistro_primario,COLISAO
qtd_gravidade_fatal,5
qtd_gravidade_grave,0
qtd_gravidade_leve,0


In [62]:
anos = df['data_hora'].dt.year.unique()

feriados = holidays.Brazil(subdiv='SP', years=anos)


In [63]:
df['data'] = df['data_hora'].dt.normalize()

df['feriado'] = df['data'].dt.date.isin(feriados).astype('int')

df = df.drop(columns=['data'])

In [64]:
df['feriado'].describe()

count    29516.000000
mean         0.026901
std          0.161796
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: feriado, dtype: float64

In [65]:
print(df.duplicated().sum())

2


In [66]:
df.drop_duplicates(inplace=True)

In [67]:
"""filter_roads = {'highway': ['motorway', 'trunk', 'primary', 'secondary', 'tertiary']}

x_min = df['longitude'].min() - 0.05
x_max = df['longitude'].max() + 0.05
y_min = df['latitude'].min() - 0.05
y_max = df['latitude'].max() + 0.05

step = 3.0

x_axis = np.arange(x_min, x_max, step)
y_axis = np.arange(y_min, y_max, step)

total_steps = len(x_axis) * len(y_axis)
i = 0

parquet_list = []

print(f"{total_steps} steps")

for x in x_axis:
    
    for y in y_axis:

        i = i + 1

        current_area = [x, y, x + step, y + step]

        print(f"{i}/{total_steps}")

        try:

            sp_graph = OSM(filepath='../data_raw/sudeste-251219.osm.pbf', bounding_box=current_area)

            geodf = sp_graph.get_data_by_custom_criteria(custom_filter=filter_roads, filter_type='keep', keep_ways=True, keep_nodes=False)

            if geodf is not None and not geodf.empty:
                parquet_list.append(geodf)

            del sp_graph
            del geodf
            gc.collect()

        except Exception:
            continue

if parquet_list:

    final_map = pd.concat(parquet_list, ignore_index=True)
    final_map.to_parquet('final_map_sp.parquet')

else:
    print("Error") """

'filter_roads = {\'highway\': [\'motorway\', \'trunk\', \'primary\', \'secondary\', \'tertiary\']}\n\nx_min = df[\'longitude\'].min() - 0.05\nx_max = df[\'longitude\'].max() + 0.05\ny_min = df[\'latitude\'].min() - 0.05\ny_max = df[\'latitude\'].max() + 0.05\n\nstep = 3.0\n\nx_axis = np.arange(x_min, x_max, step)\ny_axis = np.arange(y_min, y_max, step)\n\ntotal_steps = len(x_axis) * len(y_axis)\ni = 0\n\nparquet_list = []\n\nprint(f"{total_steps} steps")\n\nfor x in x_axis:\n\n    for y in y_axis:\n\n        i = i + 1\n\n        current_area = [x, y, x + step, y + step]\n\n        print(f"{i}/{total_steps}")\n\n        try:\n\n            sp_graph = OSM(filepath=\'../data_raw/sudeste-251219.osm.pbf\', bounding_box=current_area)\n\n            geodf = sp_graph.get_data_by_custom_criteria(custom_filter=filter_roads, filter_type=\'keep\', keep_ways=True, keep_nodes=False)\n\n            if geodf is not None and not geodf.empty:\n                parquet_list.append(geodf)\n\n            de

In [68]:
sp_map = pq.ParquetFile('../data_processed/final_map_sp.parquet')

In [69]:
print(sp_map.schema.names)

['access', 'bicycle', 'bridge', 'busway', 'cycleway', 'foot', 'highway', 'junction', 'lanes', 'lit', 'maxspeed', 'motor_vehicle', 'name', 'oneway', 'overtaking', 'ref', 'sidewalk', 'smoothness', 'surface', 'tracktype', 'tunnel', 'width', 'id', 'timestamp', 'version', 'tags', 'osm_type', 'geometry', 'area', 'bicycle_road', 'footway', 'int_ref', 'motorcar', 'motorroad', 'psv', 'service', 'segregated', 'turn', 'changeset']


In [70]:
first_chunk = sp_map.read_row_group(0)

parquet_view = first_chunk.to_pandas()

print(parquet_view.head())

  access bicycle bridge busway cycleway  foot    highway junction lanes   lit  \
0   None    None   None   None     None  None   motorway     None  None  None   
1   None    None   None   None     None  None   tertiary     None  None  None   
2   None    None   None   None     None  None  secondary     None     2  None   
3   None    None   None   None     None  None  secondary     None     2  None   
4   None    None   None   None     None  None  secondary     None     2  None   

   ... bicycle_road footway int_ref motorcar motorroad   psv service  \
0  ...         None    None    None     None      None  None    None   
1  ...         None    None    None     None      None  None    None   
2  ...         None    None    None     None      None  None    None   
3  ...         None    None    None     None      None  None    None   
4  ...         None    None    None     None      None  None    None   

  segregated  turn changeset  
0       None  None       NaN  
1       None  None

In [71]:
print(parquet_view.isna().mean())

access           0.994252
bicycle          0.963833
bridge           0.942072
busway           0.999945
cycleway         0.997322
foot             0.955417
highway          0.000000
junction         0.952479
lanes            0.472488
lit              0.866931
maxspeed         0.640015
motor_vehicle    0.998789
name             0.196361
oneway           0.281619
overtaking       0.999010
ref              0.733704
sidewalk         0.968664
smoothness       0.993026
surface          0.199439
tracktype        0.996544
tunnel           0.998279
width            0.996955
id               0.000000
timestamp        0.000000
version          0.000000
tags             0.000000
osm_type         0.000000
geometry         0.000000
area             0.999982
bicycle_road     0.999971
footway          0.999963
int_ref          0.999978
motorcar         0.999872
motorroad        0.999453
psv              0.999956
service          0.999993
segregated       0.999938
turn             0.999967
changeset   

In [72]:
parquet_view['bridge'].value_counts()

bridge
yes         13140
viaduct      2646
aqueduct        2
trestle         1
bridge          1
damaged         1
Name: count, dtype: int64

In [73]:
parquet_view['tunnel'].value_counts()

tunnel
yes                 426
building_passage     40
passage               3
Name: count, dtype: int64

In [74]:
parquet_view['id'].value_counts()

id
498620825     3
187785144     2
546926273     2
339957360     2
604393893     2
             ..
1459522461    1
1459522462    1
1459522463    1
1459522464    1
31585447      1
Name: count, Length: 272302, dtype: int64

In [75]:
geometry = gpd.points_from_xy(df['longitude'], df['latitude'])

gdf_accidents = gpd.GeoDataFrame(df, geometry=geometry  )

gdf_accidents.set_crs(epsg=4326, inplace=True)

gdf_accidents = gdf_accidents.to_crs(epsg=31983)

In [76]:
columns_parquet = ['id', 'geometry', 'highway', 'oneway', 'lanes', 'maxspeed', 'bridge', 'tunnel', 'lit', 'surface', 'junction']

geodf = gpd.read_parquet('../data_processed/final_map_sp.parquet', columns=columns_parquet)

geodf = geodf.to_crs(epsg=31983)

In [77]:
gdf = gpd.sjoin_nearest(gdf_accidents, geodf, how='left', max_distance=1000, distance_col='dist_imprecisao')

In [78]:
gdf['dist_imprecisao'].describe()

count    29541.000000
mean        23.210714
std         78.457090
min          0.000074
25%          1.336256
50%          3.218376
75%          9.042495
max        948.887008
Name: dist_imprecisao, dtype: float64

In [79]:
mask = gdf['dist_imprecisao'] > 50

print(len(df[mask]))

2557


C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_18908\2600821881.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(len(df[mask]))


In [80]:
gdf = gdf[~mask]

In [81]:
gdf['dist_imprecisao'].describe()

count    26984.000000
mean         5.906687
std          8.332170
min          0.000074
25%          1.205211
50%          2.826240
75%          6.487523
max         49.936006
Name: dist_imprecisao, dtype: float64

In [82]:
gdf['id'].isna().sum() 

np.int64(74)

In [83]:
gdf['id'].info()

<class 'pandas.core.series.Series'>
Index: 27058 entries, 11 to 755855
Series name: id
Non-Null Count  Dtype  
--------------  -----  
26984 non-null  float64
dtypes: float64(1)
memory usage: 422.8 KB


In [84]:
gdf['id'] = gdf['id'].fillna(-1)
gdf['id'] = gdf['id'].astype('int')

In [85]:
gdf.drop(columns='index_right', inplace=True)

In [86]:
gdf['highway'].value_counts(dropna=False)

highway
motorway     21610
trunk         2348
secondary     1157
primary        955
tertiary       914
NaN             74
Name: count, dtype: int64

In [87]:
gdf.dropna(subset=['highway'], inplace=True)

In [88]:
mask = (gdf['highway'] == 'tertiary') | (gdf['highway'] == 'secondary') | (gdf['highway'] == 'primary')

gdf = gdf[~mask]

In [89]:
gdf['oneway'].value_counts(dropna=False)

oneway
yes     23543
no        293
None      122
Name: count, dtype: int64

In [90]:
mask = gdf['oneway'].isnull()

df_temp = gdf.loc[mask, 'highway']

print(df_temp.value_counts())

highway
trunk    122
Name: count, dtype: int64


In [91]:
gdf['lanes'].value_counts(dropna=False)

lanes
2       11860
3        7868
4        2064
5         801
None      699
11        189
6         169
9         112
1          69
10         31
18         24
14         19
8          18
12         12
7          10
20          6
15          4
21          3
Name: count, dtype: int64

In [92]:
gdf['lanes'].info()

<class 'pandas.core.series.Series'>
Index: 23958 entries, 11 to 755855
Series name: lanes
Non-Null Count  Dtype 
--------------  ----- 
23259 non-null  object
dtypes: object(1)
memory usage: 374.3+ KB


In [93]:
gdf['lanes'] = pd.to_numeric(gdf['lanes'], errors='coerce')

In [94]:
mask = gdf['lanes'] > 18

print(gdf.loc[mask, ['lanes', 'highway', 'oneway']])

        lanes   highway oneway
71542    20.0  motorway    yes
148525   20.0  motorway    yes
339524   20.0  motorway    yes
391218   20.0  motorway    yes
408539   20.0  motorway    yes
422694   21.0  motorway    yes
483041   21.0  motorway    yes
501099   20.0  motorway    yes
727924   21.0  motorway    yes


In [95]:
mask = gdf['lanes'] == 18

print(gdf.loc[mask, ['lanes', 'highway', 'oneway']])

        lanes   highway oneway
13489    18.0  motorway    yes
30829    18.0  motorway    yes
142561   18.0  motorway    yes
150397   18.0  motorway    yes
159274   18.0  motorway    yes
161955   18.0  motorway    yes
249391   18.0  motorway    yes
262507   18.0  motorway    yes
305372   18.0  motorway    yes
318694   18.0  motorway    yes
350974   18.0  motorway    yes
357134   18.0  motorway    yes
376539   18.0  motorway    yes
386442   18.0  motorway    yes
407338   18.0  motorway    yes
419988   18.0  motorway    yes
427540   18.0  motorway    yes
509598   18.0  motorway    yes
517510   18.0  motorway    yes
532956   18.0  motorway    yes
540377   18.0  motorway    yes
556888   18.0  motorway    yes
564977   18.0  motorway    yes
589218   18.0  motorway    yes


In [96]:
mask = (gdf['lanes'] >= 14) & (gdf['lanes'] < 18)

print(gdf.loc[mask, ['lanes', 'highway', 'oneway']])

        lanes   highway oneway
3855     14.0  motorway    yes
15456    14.0  motorway    yes
44320    14.0  motorway    yes
47955    14.0  motorway    yes
106455   14.0  motorway    yes
138778   14.0  motorway    yes
165024   14.0  motorway    yes
201156   14.0  motorway    yes
219662   14.0  motorway    yes
266735   14.0  motorway    yes
283281   14.0  motorway    yes
283768   14.0  motorway    yes
288241   14.0  motorway    yes
321269   14.0  motorway    yes
328808   15.0  motorway    yes
372901   14.0  motorway    yes
468428   14.0  motorway    yes
496032   15.0  motorway    yes
502350   15.0  motorway    yes
536180   14.0  motorway    yes
585578   14.0  motorway    yes
610538   14.0  motorway    yes
611639   15.0  motorway    yes


In [97]:
mask = (gdf['lanes'] >= 5) & (gdf['oneway'] != 'yes') 

print(gdf.loc[mask, ['lanes', 'highway', 'oneway']])

Empty DataFrame
Columns: [lanes, highway, oneway]
Index: []


In [98]:
gdf['maxspeed'].value_counts(dropna=False)

maxspeed
100     6664
110     4761
120     4299
90      4071
80      2012
60       804
40       625
None     274
50       226
70       198
30        14
35        10
Name: count, dtype: int64

In [99]:
gdf['maxspeed'].info()

<class 'pandas.core.series.Series'>
Index: 23958 entries, 11 to 755855
Series name: maxspeed
Non-Null Count  Dtype 
--------------  ----- 
23684 non-null  object
dtypes: object(1)
memory usage: 374.3+ KB


In [100]:
gdf['maxspeed'] = gdf['maxspeed'].replace(to_replace='50; 30; 30', value='50')

In [101]:
gdf['maxspeed'] = pd.to_numeric(gdf['maxspeed'], errors='coerce')

In [102]:
gdf['maxspeed'].value_counts(dropna=False)

maxspeed
100.0    6664
110.0    4761
120.0    4299
90.0     4071
80.0     2012
60.0      804
40.0      625
NaN       274
50.0      226
70.0      198
30.0       14
35.0       10
Name: count, dtype: int64

In [103]:
mask = gdf['concessionaria'].notna()

print(gdf.loc[mask, 'maxspeed'].value_counts())

maxspeed
100.0    6664
110.0    4761
120.0    4299
90.0     4071
80.0     2012
60.0      804
40.0      625
50.0      226
70.0      198
30.0       14
35.0       10
Name: count, dtype: int64


In [104]:
mask = gdf['maxspeed'].isna()

print(gdf.loc[mask, 'concessionaria'].value_counts())

concessionaria
CART                      100
DER                        80
ECOPISTAS                  57
NAO DISPONIVEL             18
AUTOBAN                     6
ECOVIAS RAPOSO-CASTELO      5
ECONOROESTE                 4
ENTREVIAS                   2
VIAOESTE                    1
SPVIAS                      1
Name: count, dtype: int64


In [105]:
gdf['bridge'].value_counts(dropna=False)

bridge
None       23207
yes          505
viaduct      246
Name: count, dtype: int64

In [106]:
gdf['bridge'] = gdf['bridge'].fillna(0)

gdf['bridge'] = gdf['bridge'].replace('viaduct', 1)

gdf['bridge'] = gdf['bridge'].replace('yes', 1)

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_18908\954858790.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf['bridge'] = gdf['bridge'].replace('yes', 1)


In [107]:
gdf['bridge'] = gdf['bridge'].astype('int')

In [108]:
gdf['tunnel'].value_counts(dropna=False)

tunnel
None    23955
yes         3
Name: count, dtype: int64

In [109]:
gdf = gdf.drop('tunnel', axis=1)

In [110]:
gdf['lit'].value_counts(dropna=False)

lit
None    20614
yes      3147
no        197
Name: count, dtype: int64

In [111]:
gdf = gdf.drop('lit', axis=1)

In [112]:
gdf['surface'].value_counts(dropna=False)

surface
asphalt     20157
paved        3536
concrete      148
None          117
Name: count, dtype: int64

In [113]:
surface_dict = {
    'paved': 'asphalt',
    'unpaved': 'other',
    'ground': 'other',
    'concrete': 'other',
    'sett': 'other',
    'paving_stones': 'other'
}

gdf['surface'] = gdf['surface'].replace(surface_dict)

In [114]:
gdf['junction'].value_counts(dropna=False)

junction
None          23937
circular         16
roundabout        5
Name: count, dtype: int64

In [115]:
gdf = gdf.drop('junction', axis=1)

In [116]:
columns_dropna = ['oneway', 'lanes', 'maxspeed']

gdf = gdf.dropna(subset=columns_dropna)

In [117]:
gdf['oneway'] = gdf['oneway'].replace('yes', 1)
gdf['oneway'] = gdf['oneway'].replace('no', 0)

gdf['oneway'] = gdf['oneway'].astype('int')

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_18908\3829129697.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf['oneway'] = gdf['oneway'].replace('no', 0)


In [118]:
gdf['lanes'].value_counts()

lanes
2.0     11621
3.0      7838
4.0      2061
5.0       801
11.0      189
6.0       168
9.0       112
1.0        57
10.0       31
18.0       24
14.0       19
8.0        18
12.0       12
7.0        10
20.0        6
15.0        4
21.0        3
Name: count, dtype: int64

In [119]:
mask = gdf['lanes'] > 8

gdf = gdf[~mask]

In [120]:
gdf = gdf.drop('surface', axis=1)

In [121]:
gdf['lanes'] = gdf['lanes'].astype('int')
gdf['maxspeed'] = gdf['maxspeed'].astype('int')

In [122]:
gdf['highway'] = gdf['highway'].replace('motorway', 1)
gdf['highway'] = gdf['highway'].replace('trunk', 0)

C:\Users\Arthur Rios\AppData\Local\Temp\ipykernel_18908\635258844.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf['highway'] = gdf['highway'].replace('trunk', 0)


In [123]:
gdf.rename(columns={'highway': 'motorway'}, inplace=True)

In [124]:
gdf['motorway'] = gdf['motorway'].astype('int')

In [125]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 22574 entries, 81 to 755855
Data columns (total 44 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   logradouro                       22574 non-null  object        
 1   municipio                        22574 non-null  object        
 2   latitude                         22574 non-null  float64       
 3   longitude                        22574 non-null  float64       
 4   tipo_registro                    22574 non-null  object        
 5   turno                            22574 non-null  object        
 6   tp_sinistro_primario             22574 non-null  object        
 7   qtd_gravidade_fatal              22574 non-null  int64         
 8   qtd_gravidade_grave              22574 non-null  int64         
 9   qtd_gravidade_leve               22574 non-null  int64         
 10  qtd_caminhao                     22574 non-null  int6

In [126]:
gdf.to_csv('data_processed.csv', index=False)